## Deliverable 2. Create a Customer Travel Destinations Map.

In [266]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [267]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Campbellsville,US,2021-10-16 13:36:43,37.3434,-85.3419,54.68,69,61,3.00,broken clouds
1,1,Port Elizabeth,ZA,2021-10-16 13:33:49,-33.9180,25.5701,70.11,45,0,31.07,clear sky
2,2,Mehamn,NO,2021-10-16 13:36:43,71.0357,27.8492,39.27,84,100,24.52,light rain
3,3,Busselton,AU,2021-10-16 13:29:17,-33.6500,115.3333,61.09,74,94,10.27,overcast clouds
4,4,Bambous Virieux,MU,2021-10-16 13:36:44,-20.3428,57.7575,77.25,65,75,16.11,broken clouds


In [268]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
## Input statements are written to prompt the customer for their minimum and maximum temperature preferences. (5 pt) ##
min_temp=int(input('Enter the minimum vacation temperature desired [deg F]'))
max_temp=int(input('Enter the maximum vacation temperature desired [deg F]'))

In [269]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

city_data_df=city_data_df.loc[(city_data_df['Max Temp']<max_temp) & (city_data_df['Max Temp']>min_temp)]
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Port Elizabeth,ZA,2021-10-16 13:33:49,-33.9180,25.5701,70.11,45,0,31.07,clear sky
4,4,Bambous Virieux,MU,2021-10-16 13:36:44,-20.3428,57.7575,77.25,65,75,16.11,broken clouds
6,6,Rondonopolis,BR,2021-10-16 13:36:45,-16.4708,-54.6356,78.94,78,100,7.70,moderate rain
7,7,Bethel,US,2021-10-16 13:36:45,41.3712,-73.4140,71.01,83,1,1.01,clear sky
10,10,Albany,US,2021-10-16 13:30:02,42.6001,-73.9662,70.66,88,100,1.01,overcast clouds


In [270]:
city_data_df.count()

City_ID                234
City                   234
Country                232
Date                   234
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Current Description    234
dtype: int64

In [271]:
# 4a. Determine if there are any empty rows.
city_data_df.isnull().sum()

City_ID                0
City                   0
Country                2
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [272]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
## A new DataFrame is created based on the minimum and maximum temperature, and empty rows are dropped. (5 pt) ##
try:
    clean_df=city_data_df.dropna()
    
except:
    print('No empty rows to drop')
    clean_df=city_data_df
  

In [273]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Elizabeth,ZA,70.11,clear sky,-33.9180,25.5701,
4,Bambous Virieux,MU,77.25,broken clouds,-20.3428,57.7575,
6,Rondonopolis,BR,78.94,moderate rain,-16.4708,-54.6356,
7,Bethel,US,71.01,clear sky,41.3712,-73.4140,
10,Albany,US,70.66,overcast clouds,42.6001,-73.9662,
11,Beloha,MG,84.60,clear sky,-25.1667,45.0500,
12,Butaritari,KI,81.37,scattered clouds,3.0707,172.7902,
14,Katsuura,JP,73.18,scattered clouds,35.1333,140.3000,
15,Rikitea,PF,74.30,few clouds,-23.1203,-134.9692,
22,Ponta Do Sol,PT,83.71,scattered clouds,32.6667,-17.1000,


In [274]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": coord,
    "language": "English"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    Lat=row['Lat']
    Lng=row['Lng']
    
    coord=f"{Lat},{Lng}"

    params.update({"location":coord})

     # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # extract results
    try:
        hotel_df.loc[index,"Hotel Name"]=results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index,"Hotel Name"]=np.nan

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [275]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Elizabeth,ZA,70.11,clear sky,-33.9180,25.5701,39 On Nile Guest House
4,Bambous Virieux,MU,77.25,broken clouds,-20.3428,57.7575,Casa Tia Villa
6,Rondonopolis,BR,78.94,moderate rain,-16.4708,-54.6356,Serras Hotel
7,Bethel,US,71.01,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
10,Albany,US,70.66,overcast clouds,42.6001,-73.9662,NaN
11,Beloha,MG,84.60,clear sky,-25.1667,45.0500,NaN
12,Butaritari,KI,81.37,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
14,Katsuura,JP,73.18,scattered clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
15,Rikitea,PF,74.30,few clouds,-23.1203,-134.9692,People ThankYou
22,Ponta Do Sol,PT,83.71,scattered clouds,32.6667,-17.1000,Hotel do Campo


In [276]:
# 7. Drop the rows where there is no Hotel Name.
## The hotel name is retrieved and added to the DataFrame, and the rows that don’t have a hotel name are dropped. (10 pt) ##
try:
    clean_hotel_df=hotel_df.dropna()
    
except:
    print('No empty rows to drop')
    clean_hotel_df=hotel_df

In [277]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Port Elizabeth,ZA,70.11,clear sky,-33.9180,25.5701,39 On Nile Guest House
4,Bambous Virieux,MU,77.25,broken clouds,-20.3428,57.7575,Casa Tia Villa
6,Rondonopolis,BR,78.94,moderate rain,-16.4708,-54.6356,Serras Hotel
7,Bethel,US,71.01,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
12,Butaritari,KI,81.37,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
14,Katsuura,JP,73.18,scattered clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
15,Rikitea,PF,74.30,few clouds,-23.1203,-134.9692,People ThankYou
22,Ponta Do Sol,PT,83.71,scattered clouds,32.6667,-17.1000,Hotel do Campo
24,Lorengau,PG,81.93,scattered clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
27,Sorong,ID,82.51,overcast clouds,-0.8833,131.2500,waigo splash hotel


In [278]:
# 8a. Create the output File (CSV)
## The DataFrame is exported as a CSV file into the Vacation_Search folder and is saved as WeatherPy_vacation.csv. (5 pt) ##
output_data_file='WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [279]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [280]:
# 9. Using the template add city name, the country code, the weather description 
# and maximum temperature for the city.

info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Weather</b></dt><dd>{Current Description}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [281]:
## A marker layer map with pop-up markers for the cities in the vacation DataFrame is created, 
# and it is uploaded as a PNG. Each marker has the following information: (5 pt)

# 11a. Add a marker layer for each city to the map. 
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# 11b. Display the figure and save an image
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [265]:
dir(fig)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_cross_validation_lock',
 '_default_keys',
 '_default_layout',
 '_display_callbacks',
 '_dom_classes',
 '_errors_box',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_trait_default_generator',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_map',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_observers',
 '_notify_trait',
 '_property_l